# Gesture Recognition

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [12]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [13]:
train_doc = np.random.permutation(open('./train.csv').readlines())
val_doc = np.random.permutation(open('./val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with some of the parts of the generator function such that you get high accuracy.

In [14]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,18,84,84,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,18,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(84,84)).astype(np.float32)
                    else:
                        image = imresize(image,(84,84)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [15]:
curr_dt_time = datetime.datetime.now()
train_path = './train'
val_path = './val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D`. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

In [16]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [17]:
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_5 (Batch (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_5 (Activation)    (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_6 (Batch (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_6 (Activation)    (None, 9, 42, 84, 128)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [18]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [21]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./val ; batch size = 10
Source path =  ./train ; batch size = 10
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 3s - loss: 3.1421 - categorical_accuracy: 0.2938

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 118s 2s/step - loss: 3.1196 - categorical_accuracy: 0.2881 - val_loss: 1.9488 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2019-09-2315_47_36.115999/model-00001-3.12791-0.29110-1.94879-0.30000.h5
Epoch 2/30
67/67 [==============================] - 64s 961ms/step - loss: 1.4271 - categorical_accuracy: 0.4483 - val_loss: 1.0217 - val_categorical_accuracy: 0.6500

Epoch 00002: saving model to model_init_2019-09-2315_47_36.115999/model-00002-1.42855-0.44947-1.02168-0.65000.h5
Epoch 3/30
57/67 [========================>.....] - ETA: 7s - loss: 1.2432 - categorical_accuracy: 0.4825

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 55s 816ms/step - loss: 1.2432 - categorical_accuracy: 0.4831 - val_loss: 1.0680 - val_categorical_accuracy: 0.6100

Epoch 00003: saving model to model_init_2019-09-2315_47_36.115999/model-00003-1.24235-0.48115-1.06799-0.61000.h5
Epoch 4/30
67/67 [==============================] - 55s 814ms/step - loss: 1.1308 - categorical_accuracy: 0.5134 - val_loss: 0.9496 - val_categorical_accuracy: 0.7000

Epoch 00004: saving model to model_init_2019-09-2315_47_36.115999/model-00004-1.13534-0.50830-0.94959-0.70000.h5
Epoch 5/30
67/67 [==============================] - 54s 813ms/step - loss: 1.0124 - categorical_accuracy: 0.5901 - val_loss: 0.9846 - val_categorical_accuracy: 0.5300

Epoch 00005: saving model to model_init_2019-09-2315_47_36.115999/model-00005-1.00585-0.59276-0.98458-0.53000.h5
Epoch 6/30
67/67 [==============================] - 54s 812ms/step - loss: 1.0505 - categorical_accuracy: 0.5696 - val_loss: 0.8201 - val_categorical_accuracy: 0.7300



In [23]:
from keras.models import load_model, Model

model_name = './model_init_2019-09-2315_47_36.115999/model-00030-0.33385-0.87632-0.54638-0.73000.h5'
    
test_doc = open('./val.csv').readlines()
test_path = './val'
num_test_sequences = len(test_doc)
print ('# testing sequences =', num_test_sequences)
test_generator = generator(test_path, test_doc, batch_size)
model = load_model(model_name)
print("Model loaded.")
model_func = Model(inputs=[model.input], outputs=[model.output])
    
acc = 0
    
num_batches = int(num_test_sequences/batch_size)
    
for i in range(num_batches):
    x,true_labels = test_generator.__next__()
    print ("shape of x:", x.shape, "and shape of true_labels:", true_labels.shape)
    pred_idx = np.argmax(model_func.predict_on_batch(x), axis=1)
    for j,k in enumerate(pred_idx):
        if true_labels[j,k] == 1:
            acc += 1
                
if (num_test_sequences%batch_size) != 0:
    x,true_labels = test_generator.__next__()
    print ("shape of x:", x.shape, "and shape of true_labels:", true_labels.shape)
    pred_idx = np.argmax(model_func.predict_on_batch(x), axis=1)
    for j,k in enumerate(pred_idx):
        if true_labels[j,k] == 1:
            acc += 1

print('Accuracy is =', acc/num_test_sequences) 


# testing sequences = 100
Model loaded.
Source path =  ./val ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
shape of x: (10, 18, 84, 84, 3) and shape of true_labels: (10, 5)
Accuracy is = 0.73


#### Training model with batch size = 20 and epoch = 30

In [24]:
batch_size = 20
epoch = 30
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  from ipykernel import kernelapp as app


34/34 [==============================] - 31s 906ms/step - loss: 0.3122 - categorical_accuracy: 0.8765 - val_loss: 0.5681 - val_categorical_accuracy: 0.7800

Epoch 00001: saving model to model_init_2019-09-2315_47_36.115999/model-00001-0.31223-0.87647-0.56813-0.78000.h5
Epoch 2/30
12/34 [=========>....................] - ETA: 16s - loss: 0.3226 - categorical_accuracy: 0.8583

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 27s 796ms/step - loss: 0.3300 - categorical_accuracy: 0.8529 - val_loss: 0.5508 - val_categorical_accuracy: 0.7200

Epoch 00002: saving model to model_init_2019-09-2315_47_36.115999/model-00002-0.33378-0.84985-0.55078-0.72000.h5
Epoch 3/30
34/34 [==============================] - 28s 819ms/step - loss: 0.3040 - categorical_accuracy: 0.8882 - val_loss: 0.4457 - val_categorical_accuracy: 0.7800

Epoch 00003: saving model to model_init_2019-09-2315_47_36.115999/model-00003-0.30400-0.88824-0.44575-0.78000.h5
Epoch 4/30
34/34 [==============================] - 27s 804ms/step - loss: 0.3195 - categorical_accuracy: 0.8617 - val_loss: 0.7260 - val_categorical_accuracy: 0.7200

Epoch 00004: saving model to model_init_2019-09-2315_47_36.115999/model-00004-0.32168-0.85886-0.72600-0.72000.h5
Epoch 5/30
34/34 [==============================] - 28s 819ms/step - loss: 0.2939 - categorical_accuracy: 0.8676 - val_loss: 0.5155 - val_categorical_accuracy: 0.7800



34/34 [==============================] - 27s 801ms/step - loss: 0.2514 - categorical_accuracy: 0.9264 - val_loss: 0.5562 - val_categorical_accuracy: 0.8200

Epoch 00030: saving model to model_init_2019-09-2315_47_36.115999/model-00030-0.25494-0.92492-0.55623-0.82000.h5


In [29]:
from keras.applications.vgg16 import VGG16
from keras.layers import TimeDistributed

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
x = base_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base_model.input, outputs=features)
    
for layer in base_model.layers:
    layer.trainable = False
        
model = Sequential()
model.add(TimeDistributed(conv_model, input_shape=(18,84,84,3)))
model.add(GRU(32, return_sequences=True))
model.add(GRU(16))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 18, 64)            14845824  
_________________________________________________________________
gru_3 (GRU)                  (None, 18, 32)            9312      
_________________________________________________________________
gru_4 (GRU)                  (None, 16)                2352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 45        
Total params: 14,857,669
Trainable params: 142,981
Non-trainable params: 14,714,688
__________________________________________________________

In [30]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [31]:
model_name = 'model_init_conv_lstm' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Source path =  ./val ; batch size = 20
Source path =  ./train ; batch size = 20
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/34 [===========================>..] - ETA: 2s - loss: 1.6577 - categorical_accuracy: 0.2063

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 53s 2s/step - loss: 1.6531 - categorical_accuracy: 0.2016 - val_loss: 1.5967 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_conv_lstm_2019-09-2315_47_36.115999/model-00001-1.65422-0.20664-1.59666-0.20000.h5
Epoch 2/30
34/34 [==============================] - 44s 1s/step - loss: 1.6243 - categorical_accuracy: 0.2333 - val_loss: 1.5777 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_conv_lstm_2019-09-2315_47_36.115999/model-00002-1.62779-0.23077-1.57774-0.22000.h5
Epoch 3/30
34/34 [==============================] - 45s 1s/step - loss: 1.6005 - categorical_accuracy: 0.2046 - val_loss: 1.5710 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to model_init_conv_lstm_2019-09-2315_47_36.115999/model-00003-1.59604-0.20965-1.57102-0.29000.h5
Epoch 4/30
34/34 [==============================] - 45s 1s/step - loss: 1.5652 - categorical_accuracy: 0.2575 - val_loss: 1.5711 - val_categorical_

34/34 [==============================] - 45s 1s/step - loss: 1.0324 - categorical_accuracy: 0.6630 - val_loss: 1.3890 - val_categorical_accuracy: 0.4100

Epoch 00029: saving model to model_init_conv_lstm_2019-09-2315_47_36.115999/model-00029-1.03083-0.65460-1.38897-0.41000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 30/30
34/34 [==============================] - 45s 1s/step - loss: 1.0289 - categorical_accuracy: 0.6497 - val_loss: 1.3895 - val_categorical_accuracy: 0.4300

Epoch 00030: saving model to model_init_conv_lstm_2019-09-2315_47_36.115999/model-00030-1.02948-0.65762-1.38945-0.43000.h5


### Conclusions
* To conclude, a smaller mini-batch size (not too small) usually leads not only to a smaller number of iterations of a training algorithm, than a large batch size, but also to a higher accuracy overall, i.e, a neural network that performs better, in the same amount of training time, or less. Here we find the optimal batch size as 20. Whereas batch size 10 is too small and also need more time for training.
* GRU is better than LSTM as it is easy to modify and doesn't need memory units, therefore, faster to train than LSTM and give as per performance.
* Conv3D gives better accuracy when compared with LSTM/GRU models
* We can **select the Conv3D model with 82% accuracy as our final model for Gesture Recognition**.